# Introduction

This notebook is for having doing inference. The best models and weights are saved in the folder. just running all the notebooks will generate the final predictions csvs

# 0. Setup

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
!pip install -r "requirement (1).txt"

### utility libraries

In [3]:
import os
import glob
import pathlib
from IPython import display
from pathlib import Path
from IPython.display import Audio
import matplotlib.pyplot as plt

# data handling libraries
import numpy as np
import librosa
import librosa.display
import pandas as pd
from sklearn.model_selection import train_test_split

# ML and DL Libraries
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import KFold

### Declaring Hyperparameters

In [4]:
seed = 42
SR = 16000 #sampling rate of each audio file
BATCH_SIZE = 1
AUD_LENGTH = 10# we are taking 10s audio for each sample to have an uniform dataset
TRAIN_TEST_SPLIT = 0.3

### All file paths

In [5]:
# # insert needed paths here
# # this is the known and unknown dataset path
# CLASSWISE_DATASET_PATH = '/content/drive/MyDrive/resnet1D_raw_wave_submission/Dataset/classwise_dataset/'

# # this is the random extra data addition folders
# ASVSPOOF_DATA_PATH  = '/content/drive/MyDrive/resnet1D_raw_wave_submission/Dataset/external_data_sources/asvspoof19_1000'
# LIBRISPEECH_DATA_PATH = '/content/drive/MyDrive/resnet1D_raw_wave_submission/Dataset/external_data_sources/librispeech_1000'

# this is the evaluation folder for phase 1 and 2
EVAL_PATH_1 = 'spcup_2022_eval_part1'
EVAL_PATH_2 = 'spcup_2022_eval_part2'

# loading paths 

MODEL_WEIGHTS_FOLDER = 'best'
# ### saving paths
# CSV_DIR = '/content/drive/MyDrive/resnet1D_raw_wave_submission/Output_files/custom_training'
# MODEL_SAVE_DIR = '/content/drive/MyDrive/resnet1D_raw_wave_submission/Output_files/custom_training'
# WEIGHT_SAVE_DIR = '/content/drive/MyDrive/resnet1D_raw_wave_submission/Output_files/custom_training'

# 1. Necessary Functions

In [6]:
# utility functions for repeating audio files
def repeated_data(file_path):
    """ This function will take a file path and give out truncated and padded to 10s version waveform"""
    y, sr = librosa.load(file_path,sr=SR)
    aud_length = AUD_LENGTH*sr # making all audio length 10 s and truncating the rest
    duration = librosa.get_duration(y=y, sr=sr)
    if duration < AUD_LENGTH:
        y = np.tile(y, int((aud_length/sr) // duration)+1)
    y = librosa.resample(y[:aud_length], orig_sr=sr, target_sr=SR)
    return y

def repeated_dataset(dataset):
    """ This function generated waveshape dataset"""
    new_ds = []
    for f in dataset:
        new_ds.append(repeated_data(f))
    return new_ds

In [7]:
def ensemble_predictions(model_weights_folder, testX,testy=1):
    cv_folders = os.listdir(MODEL_WEIGHTS_FOLDER)   
    
    model_history = list()

    for i in range(len(cv_folders)//2):
        model_name = os.path.join(MODEL_WEIGHTS_FOLDER,f"model_TSSDnet_asvspoof_cv{i}fold_.h5")
        weight_name = os.path.join(MODEL_WEIGHTS_FOLDER,f"weight_TSSDnet_asvspoof_normal_cv{i}fold_.h5")
        
        model = tf.keras.models.load_model(model_name)
        model.load_weights(weight_name)
        
        model_history.append(model)
        
    print(len(testX))
    yhats = [model.predict(testX) for model in model_history]
    yhats = np.array(yhats)
    print('yhats', yhats.shape)
    # sum across ensemble members
    summed = np.sum(yhats, axis=0)
    print('summed', summed.shape)
    # argmax across classes
    result = np.argmax(summed, axis=1)
    print('result', result.shape)
    return result

In [8]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=BATCH_SIZE, 
                 n_classes=6, shuffle=True):
        'Initialization'
        self.dim = AUD_LENGTH * SR
        self.batch_size = batch_size
        self.labels = labels
        self.shuffle = shuffle
        self.list_IDs = list_IDs
        self.on_epoch_end()

    def path_to_audio(self,path):
        """Reads and decodes an audio file."""
        return repeated_data(path)

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' 

        X = []
        y = []
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            _tempx = self.path_to_audio(self.list_IDs[ID])
            X.append(_tempx)

            # Store class
            y.append(self.labels[ID])

        return np.reshape(np.array(X), (self.batch_size,SR*AUD_LENGTH,1)).astype(np.float32),np.array(y).astype(np.float32)

# 2. Evaluation

## A. Generating prediction for phase 1 evaluation dataset¶


In [9]:
eval_path = EVAL_PATH_1
speaker_sample_paths = [
        os.path.join(eval_path, filepath)
        for filepath in os.listdir(eval_path)
        if filepath.endswith(".wav")
    ]
X_eval = []
X_eval += speaker_sample_paths

print(
    "Found {} files".format(len(X_eval))
)

Found 9000 files


In [10]:
# making a column of names
import os
n = len(X_eval)
filenames = []
for i in range(n):
    f_name, f_ext = os.path.splitext(X_eval[i])
    filenames.append(os.path.basename(f_name)+f_ext)

In [11]:
count = 0
for i in X_eval:
  if i[-7:-4]=="(1)" or i[-7:-4]=="(2)":
    count+=1
count

0

In [12]:
y_eval = [0]*len(X_eval)
print(len(X_eval),len(y_eval))

9000 9000


In [13]:
eval_ds = DataGenerator(X_eval,y_eval,shuffle=False)

In [14]:
pred_class = ensemble_predictions(MODEL_WEIGHTS_FOLDER, eval_ds)

9000
yhats (5, 9000, 6)
summed (9000, 6)
result (9000,)


In [15]:
print(len(filenames), len(pred_class))

9000 9000


In [16]:
score_df = pd.DataFrame({'file':filenames,'pred_class':pred_class})
score_df.head()

,file,pred_class
0,000564048b88c05396a9e68b3a89840e.wav,5
1,0007e065b51cb7e792b0ce301600c449.wav,1
2,001d3ce2cce8f3bb45e6023b9771d19d.wav,4
3,00260c4cfe0002f5620825ac0c03e02d.wav,1
4,002f3d510760bc5117acddbba95cf1e0.wav,5


In [17]:
# checking class distribution in our prediction
score_df["pred_class"].value_counts()

1    1604
4    1507
3    1502
0    1500
5    1488
2    1399
Name: pred_class, dtype: int64

In [18]:
# converting dataframe to csv
score_df.to_csv('part1_scores_res_asv.csv', header=False, index=False)

## B. Generating prediction for phase 2 evaluation dataset¶


In [19]:
eval_path = EVAL_PATH_2
speaker_sample_paths = [
        os.path.join(eval_path, filepath)
        for filepath in os.listdir(eval_path)
        if filepath.endswith(".wav")
    ]
X_eval = []
X_eval += speaker_sample_paths

print(
    "Found {} files".format(len(X_eval))
)

Found 9000 files


In [20]:
# making a column of names
import os
n = len(X_eval)
filenames = []
for i in range(n):
    f_name, f_ext = os.path.splitext(X_eval[i])
    filenames.append(os.path.basename(f_name)+f_ext)

In [21]:
y_eval = [1]*len(X_eval)
print(len(X_eval),len(y_eval))

9000 9000


In [22]:
eval_ds = DataGenerator(X_eval,y_eval,shuffle=False)

In [23]:
pred_class = ensemble_predictions(MODEL_WEIGHTS_FOLDER, eval_ds)

9000
yhats (5, 9000, 6)
summed (9000, 6)
result (9000,)


In [24]:
score_df = pd.DataFrame({'file':filenames,'pred_class':pred_class})
score_df.head()

,file,pred_class
0,000c8deb702043e5c1689f1e7a71950f.wav,0
1,001945bc6c04dd435ccd6780f64fac3b.wav,3
2,0020a5b1d8f42faa2fd7ff8abc942c4d.wav,4
3,00303d4b2e4bd17bd262580227ff3336.wav,1
4,00500d926ba4154bdb30b85f99dd5e6a.wav,4


In [25]:
score_df["pred_class"].value_counts()

5    1527
4    1514
1    1511
0    1498
3    1495
2    1455
Name: pred_class, dtype: int64

In [26]:
# converting dataframe to csv
score_df.to_csv('part2_scores_res_asv.csv', header=False, index=False)